In [1]:
import csv

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

RANDOM_SEED = 42

2023-04-26 15:55:38.855339: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-26 15:55:38.898953: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-26 15:55:38.899693: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-26 15:55:39.539560: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Specify each path

In [2]:
dataset = 'model/keypoint_classifier/keypoint.csv'
model_save_path = 'model/keypoint_classifier/keypoint_classifier.hdf5'
tflite_save_path = 'model/keypoint_classifier/keypoint_classifier.tflite'

# Set number of classes

In [3]:
NUM_CLASSES = 9

# Dataset reading

In [4]:
X_dataset = np.loadtxt(dataset, delimiter=',', dtype='float32', usecols=list(range(1, (21 * 2) + 1)))

In [5]:
y_dataset = np.loadtxt(dataset, delimiter=',', dtype='int32', usecols=(0))

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset, train_size=0.75, random_state=RANDOM_SEED)

# Model building

In [7]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input((21 * 2, )),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(80, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

In [8]:
model.summary()  # tf.keras.utils.plot_model(model, show_shapes=True)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout (Dropout)           (None, 42)                0         
                                                                 
 dense (Dense)               (None, 80)                3440      
                                                                 
 dropout_1 (Dropout)         (None, 80)                0         
                                                                 
 dense_1 (Dense)             (None, 10)                810       
                                                                 
 dense_2 (Dense)             (None, 9)                 99        
                                                                 
Total params: 4,349
Trainable params: 4,349
Non-trainable params: 0
_________________________________________________________________


In [9]:
# Model checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path, verbose=1, save_weights_only=False)
# Callback for early stopping
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

In [10]:
# Model compilation
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Model training

In [11]:
model.fit(
    X_train,
    y_train,
    epochs=1000,
    batch_size=128,
    validation_data=(X_test, y_test),
    callbacks=[cp_callback, es_callback]
)

Epoch 1/1000
 1/38 [..............................] - ETA: 18s - loss: 2.2016 - accuracy: 0.1641
Epoch 1: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
38/38 [==============================] - 1s 6ms/step - loss: 2.0604 - accuracy: 0.2542 - val_loss: 1.8960 - val_accuracy: 0.3192
Epoch 2/1000
 1/38 [..............................] - ETA: 0s - loss: 1.9672 - accuracy: 0.2578
Epoch 2: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
38/38 [==============================] - 0s 2ms/step - loss: 1.8267 - accuracy: 0.3221 - val_loss: 1.6570 - val_accuracy: 0.4183
Epoch 3/1000
 1/38 [..............................] - ETA: 0s - loss: 1.7834 - accuracy: 0.3281
Epoch 3: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
38/38 [==============================] - 0s 2ms/step - loss: 1.6275 - accuracy: 0.3930 - val_loss: 1.4163 - val_accuracy: 0.4833
Epoch 4/1000
 1/38 [..............................] - ETA: 0s - loss: 1.5786 - accuracy: 0.4219


 1/38 [..............................] - ETA: 0s - loss: 0.5733 - accuracy: 0.7891
Epoch 28: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
38/38 [==============================] - 0s 2ms/step - loss: 0.6124 - accuracy: 0.7742 - val_loss: 0.3288 - val_accuracy: 0.9647
Epoch 29/1000
 1/38 [..............................] - ETA: 0s - loss: 0.6265 - accuracy: 0.7656
Epoch 29: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
38/38 [==============================] - 0s 2ms/step - loss: 0.6008 - accuracy: 0.7719 - val_loss: 0.3280 - val_accuracy: 0.9634
Epoch 30/1000
 1/38 [..............................] - ETA: 0s - loss: 0.6897 - accuracy: 0.7266
Epoch 30: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
38/38 [==============================] - 0s 2ms/step - loss: 0.5952 - accuracy: 0.7793 - val_loss: 0.3178 - val_accuracy: 0.9596
Epoch 31/1000
38/38 [==============================] - ETA: 0s - loss: 0.5803 - accuracy: 0.7926
Epoch 31

34/38 [=========================>....] - ETA: 0s - loss: 0.4686 - accuracy: 0.8392
Epoch 55: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
38/38 [==============================] - 0s 3ms/step - loss: 0.4690 - accuracy: 0.8386 - val_loss: 0.2201 - val_accuracy: 0.9710
Epoch 56/1000
36/38 [===========================>..] - ETA: 0s - loss: 0.4683 - accuracy: 0.8344
Epoch 56: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
38/38 [==============================] - 0s 3ms/step - loss: 0.4692 - accuracy: 0.8332 - val_loss: 0.2275 - val_accuracy: 0.9653
Epoch 57/1000
34/38 [=========================>....] - ETA: 0s - loss: 0.4465 - accuracy: 0.8440
Epoch 57: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
38/38 [==============================] - 0s 3ms/step - loss: 0.4521 - accuracy: 0.8443 - val_loss: 0.2194 - val_accuracy: 0.9678
Epoch 58/1000
32/38 [========================>.....] - ETA: 0s - loss: 0.4713 - accuracy: 0.8311
Epoch 58

 1/38 [..............................] - ETA: 0s - loss: 0.3442 - accuracy: 0.8672
Epoch 82: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
38/38 [==============================] - 0s 2ms/step - loss: 0.3931 - accuracy: 0.8658 - val_loss: 0.1847 - val_accuracy: 0.9779
Epoch 83/1000
 1/38 [..............................] - ETA: 0s - loss: 0.3768 - accuracy: 0.8828
Epoch 83: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
38/38 [==============================] - 0s 2ms/step - loss: 0.3831 - accuracy: 0.8683 - val_loss: 0.1770 - val_accuracy: 0.9767
Epoch 84/1000
 1/38 [..............................] - ETA: 0s - loss: 0.3234 - accuracy: 0.9219
Epoch 84: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
38/38 [==============================] - 0s 2ms/step - loss: 0.3783 - accuracy: 0.8773 - val_loss: 0.1792 - val_accuracy: 0.9741
Epoch 85/1000
37/38 [============================>.] - ETA: 0s - loss: 0.3860 - accuracy: 0.8714
Epoch 85

 1/38 [..............................] - ETA: 0s - loss: 0.3221 - accuracy: 0.8828
Epoch 109: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
38/38 [==============================] - 0s 2ms/step - loss: 0.3430 - accuracy: 0.8834 - val_loss: 0.1530 - val_accuracy: 0.9767
Epoch 110/1000
 1/38 [..............................] - ETA: 0s - loss: 0.3079 - accuracy: 0.8828
Epoch 110: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
38/38 [==============================] - 0s 2ms/step - loss: 0.3497 - accuracy: 0.8807 - val_loss: 0.1645 - val_accuracy: 0.9735
Epoch 111/1000
 1/38 [..............................] - ETA: 0s - loss: 0.2652 - accuracy: 0.9141
Epoch 111: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
38/38 [==============================] - 0s 2ms/step - loss: 0.3550 - accuracy: 0.8828 - val_loss: 0.1642 - val_accuracy: 0.9760
Epoch 112/1000
 1/38 [..............................] - ETA: 0s - loss: 0.3288 - accuracy: 0.8594
Ep

Epoch 136/1000
37/38 [============================>.] - ETA: 0s - loss: 0.3154 - accuracy: 0.9001
Epoch 136: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
38/38 [==============================] - 0s 2ms/step - loss: 0.3150 - accuracy: 0.9003 - val_loss: 0.1378 - val_accuracy: 0.9785
Epoch 137/1000
 1/38 [..............................] - ETA: 0s - loss: 0.2795 - accuracy: 0.9062
Epoch 137: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
38/38 [==============================] - 0s 2ms/step - loss: 0.3111 - accuracy: 0.8961 - val_loss: 0.1472 - val_accuracy: 0.9785
Epoch 138/1000
 1/38 [..............................] - ETA: 0s - loss: 0.3352 - accuracy: 0.8750
Epoch 138: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
38/38 [==============================] - 0s 2ms/step - loss: 0.3015 - accuracy: 0.9064 - val_loss: 0.1494 - val_accuracy: 0.9798
Epoch 139/1000
38/38 [==============================] - ETA: 0s - loss: 0.3033 - accu

In [12]:
# Model evaluation
val_loss, val_acc = model.evaluate(X_test, y_test, batch_size=128)

13/13 [==============================] - 0s 866us/step - loss: 0.1576 - accuracy: 0.9760


In [13]:
# Loading the saved model
model = tf.keras.models.load_model(model_save_path)

In [14]:
# Inference test
predict_result = model.predict(np.array([X_test[0]]))
print(np.squeeze(predict_result))
print(np.argmax(np.squeeze(predict_result)))

1/1 [==============================] - 0s 58ms/step
[1.8663524e-02 9.1192031e-01 6.6212930e-02 2.1673040e-07 2.8208843e-08
 2.3854252e-08 2.7255615e-04 2.4831330e-03 4.4729246e-04]
1


# Saving Model

In [15]:
# Save as a model dedicated to inference
model.save(model_save_path, include_optimizer=False)

In [16]:
# Transform model (quantization)

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()

open(tflite_save_path, 'wb').write(tflite_quantized_model)

2023-04-26 15:55:56.604553: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'input_1' with dtype float and shape [?,42]
	 [[{{node input_1}}]]
2023-04-26 15:55:56.656437: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,42]
	 [[{{node inputs}}]]
2023-04-26 15:55:56.668050: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'input_1' with dtype float and shape [?,42]
	 [[{{node input_1}}]]
2023-04-26 15:5

INFO:tensorflow:Assets written to: /tmp/tmplkt3rx8z/assets


INFO:tensorflow:Assets written to: /tmp/tmplkt3rx8z/assets
2023-04-26 15:55:57.516081: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-04-26 15:55:57.516114: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-04-26 15:55:57.516409: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmplkt3rx8z
2023-04-26 15:55:57.517475: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-04-26 15:55:57.517490: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmplkt3rx8z
2023-04-26 15:55:57.520977: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:353] MLIR V1 optimization pass is not enabled
2023-04-26 15:55:57.521990: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2023-04-26 15:55:57.555968: I tensorflow/cc/saved_model/loader.cc:215] Running initializatio

9504

# Inference test

In [17]:
interpreter = tf.lite.Interpreter(model_path=tflite_save_path)
interpreter.allocate_tensors()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [18]:
# Get I / O tensor
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [19]:
interpreter.set_tensor(input_details[0]['index'], np.array([X_test[0]]))

In [20]:
%%time
# Inference implementation
interpreter.invoke()
tflite_results = interpreter.get_tensor(output_details[0]['index'])

CPU times: user 334 µs, sys: 19 µs, total: 353 µs
Wall time: 390 µs


In [21]:
print(np.squeeze(tflite_results))
print(np.argmax(np.squeeze(tflite_results)))

[1.9910662e-02 9.1179222e-01 6.5045729e-02 2.2865169e-07 2.8124193e-08
 2.2103707e-08 2.5026157e-04 2.5379509e-03 4.6290187e-04]
1
